In [2]:
import torch

torch.set_printoptions(threshold=torch.inf)

# 加载序列化的 .pt 文件
file_path = '/data/processed/real3dad/val/starfish_506_bulges_cut.pt'  # 替换为实际的文件路径

import torch, numpy as np, pprint

# 让 numpy / torch 打印不省略
np.set_printoptions(threshold=np.inf, linewidth=200, suppress=True)
try:
    torch.set_printoptions(profile="full")  # PyTorch 全量打印
except TypeError:
    torch.set_printoptions(threshold=float('inf'), linewidth=200)

def to_numpy(x):
    if isinstance(x, np.ndarray):
        return x
    if torch.is_tensor(x):
        return x.detach().cpu().numpy()
    return np.asarray(x)

def print_full(name, arr, as_int=False):
    a = to_numpy(arr)
    if as_int:
        a = a.astype(int)
    print(f"\n[{name}] shape={a.shape}, dtype={a.dtype}")
    # 用 array2string 强制完整展开
    print(np.array2string(a, max_line_width=200, separator=', '))

# 载入 .pt
data = torch.load(file_path, map_location='cpu')

# 元信息
print("\n[meta]")
pprint.pprint(data['meta'], width=120)

# 全量打印各字段
print_full('points', data['points'])                       # (N,3)
print_full('semantic_labels_pp', data['semantic_labels_pp'])   # (N,)
print_full('instance_labels_pp', data['instance_labels_pp'])   # (N,)
print_full('instance_masks', data['instance_masks'])           # (K,N) 布尔
print_full('semantic_labels_inst', data['semantic_labels_inst'])  # (K,)


[meta]
{'category': 'good', 'category_id': 1, 'file': '/data/Real3D-AD-PCD/starfish/test_neo/506_bulges_cut.pcd'}

[points] shape=(8192, 3), dtype=float32
[[ 93.51976   ,  -5.3781962 , 224.00967   ],
 [ 99.539665  ,  -4.2120147 , 223.58504   ],
 [105.52261   ,  -4.9838667 , 223.25664   ],
 [ 98.24444   , -11.205521  , 225.95494   ],
 [113.39655   , -15.527332  , 229.0199    ],
 [113.1842    , -16.257847  , 229.56554   ],
 [112.34736   , -16.64137   , 229.67067   ],
 [111.164314  , -16.733099  , 229.68758   ],
 [110.36507   , -16.736908  , 229.87856   ],
 [109.92969   , -16.653898  , 230.01169   ],
 [109.23501   , -16.463047  , 229.81671   ],
 [108.97547   , -16.339329  , 229.68127   ],
 [109.2711    , -16.457014  , 229.31062   ],
 [108.89019   , -16.309374  , 229.29155   ],
 [108.5365    , -16.15091   , 229.33144   ],
 [108.00467   , -15.86456   , 229.58102   ],
 [106.99055   , -15.190964  , 229.30508   ],
 [105.90332   , -14.24339   , 228.99838   ],
 [105.13462   , -13.426305  , 228.

In [4]:
# GT一致性检查
import numpy as np, torch
from pathlib import Path

root = Path("/data/processed/real3dad")
scene = "starfish_506_bulges_cut"

pt = torch.load(root/"val"/f"{scene}.pt", map_location="cpu")
N_pt = pt["points"].shape[0]

gt = np.loadtxt(root/"instance_gt"/"val"/f"{scene}.txt", dtype=np.int64)
print("N_pt =", N_pt, " N_gt =", gt.shape[0], " equal? ", N_pt == gt.shape[0])
print("unique semantic_id in GT:", np.unique(gt//1000))
print("unique inst_index in GT:", np.unique(gt%1000))

N_pt = 8192  N_gt = 8192  equal?  True
unique semantic_id in GT: [0 1]
unique inst_index in GT: [0 1 2]


In [5]:
import torch, glob, numpy as np

folders = ['train', 'val', 'test']
base = '/data/processed/real3dad'
all_files = []
for folder in folders:
    all_files.extend(glob.glob(f"{base}/{folder}/*.pt"))

label_counts = {}
bad = []
empty = 0
point_spans = {} 
instance_counts = {}

for fp in all_files:
    data = torch.load(fp, map_location='cpu')
    semantic_pp = np.asarray(data['semantic_labels_pp'])
    if len(semantic_pp) == 0:
        empty += 1
        continue
    if semantic_pp.min() < 0 or semantic_pp.max() > 1:
        bad.append(fp)
    binc = np.bincount(semantic_pp)
    counts = {label: count for label, count in enumerate(binc) if count > 0}
    if counts:
        label_counts[fp] = counts
    # 新增：统计xyz跨度
    pts = np.asarray(data['points'])  # (N,3)
    span = pts.max(axis=0) - pts.min(axis=0)  # (3,)
    point_spans[fp] = span
    # 实例数量
    semantic_labels_inst = np.asarray(data['semantic_labels_inst'])
    instance_counts[fp] = len(semantic_labels_inst)

print("empty samples:", empty)
print("label out of {0,1} files:", len(bad))

# 收集所有出现过的label，按255优先、其余降序
all_labels = set()
for counts in label_counts.values():
    all_labels.update([k for k in counts])
if 255 in all_labels:
    sorted_labels = [255] + sorted([l for l in all_labels if l != 255], reverse=True)
else:
    sorted_labels = sorted(all_labels, reverse=True)

# 文件排序，按255标签数量倒序（无255的文件视为0）
sorted_items = sorted(
    label_counts.items(),
    key=lambda x: x[1].get(255, 0),
    reverse=True
)

# 打印表头
header = "filename".ljust(60) + "".join([f"{l:>8}" for l in sorted_labels]) + "   n_inst   span_x   span_y   span_z"
print(header)
print("=" * len(header))

# 全量打印
for fp, counts in sorted_items:
    row = fp.ljust(60)
    for l in sorted_labels:
        row += f"{counts.get(l, 0):>8}"
    n_inst = instance_counts[fp]
    span = point_spans[fp]  # (3,)
    row += f"{n_inst:8d}{span[0]:9.3f}{span[1]:9.3f}{span[2]:9.3f}"
    print(row)

# 总计输出
total = {l: 0 for l in sorted_labels}
for counts in label_counts.values():
    for label, count in counts.items():
        if label in total:
            total[label] += count
print("\nTotal count for all labels across all files:")
for l in sorted_labels:
    print(f"label {l:>3}: {total[l]}")


empty samples: 0
label out of {0,1} files: 164
filename                                                           2       1       0   n_inst   span_x   span_y   span_z
/data/processed/real3dad/train/fish_336_sink_cut.pt              144       0    8048       1   27.480   25.022   10.220
/data/processed/real3dad/train/duck_586_bulge_cut.pt               0     347    7845       1   29.264   29.326   14.163
/data/processed/real3dad/train/starfish_597_sink_cut.pt          208       0    7984       1   26.362   26.098   11.426
/data/processed/real3dad/train/gemstone_300_bulge.pt               0     582    7610       1   21.689   17.324    8.780
/data/processed/real3dad/train/toffees_462_bulge_cut.pt            0     405    7787       1   32.060   34.314   13.201
/data/processed/real3dad/train/toffees_448_bulge.pt                0     378    7814       1   32.016   34.393   13.369
/data/processed/real3dad/train/diamond_525_sink.pt               558       0    7634       1   19.003   17.274  